In [72]:
import pandas as pd

def check_column_(df, column_name, total_number):
    x = df[column_name]
    x = x[~pd.isnull(x)]
    return len(x) == total_number & len(set(x)) == total_number

from collections import namedtuple

cathedra = namedtuple('cathedra', ['teacher', 'day', 'hour', 'classroom'])

def duplicated_indices_in_column(df, column_name):
    x = df[column_name]
    duplicated_indices = x[~pd.isnull(x)].index[x[~pd.isnull(x)].duplicated(keep=False)]
    return duplicated_indices

# style_this is a set of tuples (index, column)
def dump_to_html(df, style_this):
    html_string = '<table style="width:80%">'
    header = "<tr>"
    for col in df.columns:
        header += f"<th>{col}</th>"
    header += "</th></tr>"
    html_string += header
    for index, row in df.iterrows():
        html_string += "<tr>"
        for col in df.columns:
            if (index, col) in style_this:
                html_string += f'<td style="background-color:Orange;">{row[col]}</td>'
            else:
                html_string += f"<td>{row[col]}</td>" if pd.notnull(row[col]) else "<td></td>"  
        html_string += "</tr>"
    html_string += "</table>"
    return html_string

def wite_to_html_file(html, filename):
    with open(filename, "w") as f:
        f.write(html)

def permute(df, cathedra_1, cathedra_2 ):
    classroom_1 = cathedra_1.classroom
    df.loc[df['teacher'] == cathedra_1.teacher, cathedra_1.day + str(cathedra_1.hour)] = cathedra_2.classroom
    df.loc[df['teacher'] == cathedra_2.teacher, cathedra_2.day + str(cathedra_2.hour)] = classroom_1
    # Check if there are duplicates
    # first column
    fist_column_name = cathedra_1.day + str(cathedra_1.hour)
    second_column_name = cathedra_2.day + str(cathedra_2.hour)
    first_column_duplicated_indices = duplicated_indices_in_column(df, fist_column_name)
    second_column_duplicated_indices = duplicated_indices_in_column(df, second_column_name)
    first_column_names = df['teacher'][first_column_duplicated_indices].values
    second_column_names = df['teacher'][second_column_duplicated_indices].values
    return first_column_duplicated_indices, first_column_names, second_column_duplicated_indices, second_column_names

day_dico = {
    'LUN': 0,
    'MAR': 1,
    'MER': 2,
    'GIO': 3,
    'VEN': 4,
}

hour_dico = {
    8: 0,
    9: 1,
    10: 2,
    11: 3,
    12: 4,
    13: 5,
    14: 6
}

import re

def parse_docenti_section(file_path):
    docenti = []
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    # Find the _docenti section
    match = re.search(r'_docenti\s*\{([^}]*)\}', content, re.DOTALL)
    if not match:
        return docenti
    section = match.group(1)
    # Split by lines and parse each line
    for line in section.strip().split('\n'):
        if not line.strip():
            continue
        # Split by tab or semicolon
        fields = re.split(r'\t|;', line)
        entry = {}
        for field in fields:
            if '=' in field:
                key, value = field.split('=', 1)
                entry[key.strip()] = value.strip().strip('"')
        if entry:
            docenti.append(entry)
    idx_docenti_dico = {
        doc['idx']: doc['nome'].strip() for doc in docenti
    }
    docenti_idx_dico = {
        doc['nome'].strip(): doc['idx'] for doc in docenti
    }
    return idx_docenti_dico, docenti_idx_dico

def parse_materie_section(file_path):
    materie = []
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    # Find the _materie section
    match = re.search(r'_materie\s*\{([^}]*)\}', content, re.DOTALL)
    if not match:
        return materie
    section = match.group(1)
    # Split by lines and parse each line
    for line in section.strip().split('\n'):
        if not line.strip():
            continue
        # Split by tab or semicolon
        fields = re.split(r'\t|;', line)
        entry = {}
        for field in fields:
            if '=' in field:
                key, value = field.split('=', 1)
                entry[key.strip()] = value.strip().strip('"')
        if entry:
            materie.append(entry)

    idx_materia_dico = {
        m['idx']: m['nome'] for m in materie
    }
    materia_idx_dico = {
        m['nome']: m['idx'] for m in materie
    }
    return idx_materia_dico, materia_idx_dico

def parse_classi_section(file_path):
    classi = []
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    # Find the _classi section
    match = re.search(r'_classi\s*\{([^}]*)\}', content, re.DOTALL)
    if not match:
        return classi
    section = match.group(1)
    # Split by lines and parse each line
    for line in section.strip().split('\n'):
        if not line.strip():
            continue
        # Split by tab or semicolon
        fields = re.split(r'\t|;', line)
        entry = {}
        for field in fields:
            if '=' in field:
                key, value = field.split('=', 1)
                entry[key.strip()] = value.strip().strip('"')
        if entry:
            classi.append(entry)
    
    idx_class_dico = {
        cl['idx']: cl['nome'] for cl in classi
    }
    class_idx_dico = {
        cl['nome']: cl['idx'] for cl in classi
    }
    return idx_class_dico, class_idx_dico

from collections import defaultdict

def parse_docenti_materie(ds):
    profs = ds['Identificativo'].unique()
    content = {
        ' '.join(tid.split()[0:-1]): defaultdict(list) for tid in profs
    }
    for _, row in ds.iterrows():
        teacher_id = ' '.join(row['Identificativo'].split()[0:-1])
        number_of_classes = int(row['N.Ore']) if 'N.Ore' in row and pd.notnull(row['N.Ore']) else 0
        content[teacher_id][row['Classi']].extend(number_of_classes*[row['Materia']])
    return content

def materia_given_class_and_teacher(docenti_materie_dico, class_name, teacher_name):
    if teacher_name not in docenti_materie_dico:
        print(f"Teacher {teacher_name} not found.")
        return None
    if class_name not in docenti_materie_dico[teacher_name]:
        print(f"Class {class_name} not found for teacher {teacher_name}.")
        return None
    if len(docenti_materie_dico[teacher_name][class_name]) == 0:
        print(f"No subjects left for class {class_name} and teacher {teacher_name}.")
        return None
    return docenti_materie_dico[teacher_name][class_name].pop()

def write_down_activities(docenti_materie_dico, df_horario, name_to_idx, class_to_idx, materia_to_idx):
    activities = []
    template = 'idx={idx}	materia={materia}	no_giorni_successivi=0	utilizzo_str=1	classi=({class_id})	gruppi=(-1)	docenti=({teacher_id});'
    for idx, row in df_horario.iterrows():
        teacher_name = row['teacher']
        teacher_id = name_to_idx.get(teacher_name, None)
        if teacher_id is None:
            print(f"Warning: Teacher '{teacher_name}' not found in name_to_idx mapping.")
            continue
        for day in ['LUN', 'MAR', 'MER', 'GIO', 'VEN']:
            for hour in range(8, 14):
                classroom = row.get(f'{day}{hour}', None)
                if pd.notnull(classroom):
                    class_id = class_to_idx.get(classroom, None)
                    if class_id is None:
                        print(f"Warning: Class '{classroom}' not found in class_to_idx mapping.")
                        continue
                    materia = materia_given_class_and_teacher(docenti_materie_dico, classroom, teacher_name)
                    materia = materia_to_idx.get(materia, None)
                    activity = template.format(idx=idx, materia=materia, class_id=class_id, teacher_id=teacher_id)
                    activities.append(activity)

    return activities

def write_down_schedule(df, activities, dico_hour, dico_day):
    schedule = []
    template = 'attivita={idx}	durata=1	giorno=0{day}	ora={hour};' 
    for idx, row in df.iterrows():
        for day in ['LUN', 'MAR', 'MER', 'GIO', 'VEN']:
            for hour in range(8, 14):
                classroom = row.get(f'{day}{hour}', None)
                if pd.notnull(classroom):
                    activity_id = idx
                    day_id = dico_day[day]
                    schedule_entry = template.format(activity_id=activity_id, day=day_id, hour=hour-8)
                    schedule.append(schedule_entry)
    return schedule

In [73]:
# cathedra_1 = cathedra(teacher='BARBO', day='LUN', hour=8, classroom='4I')
# cathedra_2 = cathedra(teacher='BARBO', day='LUN', hour=13, classroom='5I')
# string = 'idx=0	materia=11	no_giorni_successivi=0	utilizzo_str=1	classi=({class_id})	gruppi=(-1)	docenti=({teacher_id});'
# string.format(class_id='1', teacher_id=2)

import pandas as pd

path = './data/orario sab.csv'
df = pd.read_csv(path)
dm = parse_docenti_materie(pd.read_csv('./data/docente_classes.csv'))
name_to_idx, _ = parse_docenti_section('./data/PATN01000Q-PROVA.drw')
class_to_idx, _ = parse_classi_section('./data/PATN01000Q-PROVA.drw')
materia_to_idx, _ = parse_materie_section('./data/PATN01000Q-PROVA.drw')
activities = write_down_activities(dm, df, name_to_idx, class_to_idx, materia_to_idx)

In [70]:
acxtivities

NameError: name 'acxtivities' is not defined